## Importing Libraries

In [3]:
import mediapipe as mp
import cv2
import numpy as np
import torch

## Virtual Try-On System 

In [4]:
def segmentor(image_path, image_type):
    # Load the user image
    if image_type == 'user_image':
        img = cv2.imread(image_path)
    else:
        img = image_path

    # Preprocessing
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)  # Apply Gaussian blur

    # Segmentation (example using GrabCut)
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (50, 50, 450, 450)  # Initial rectangle for GrabCut
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    img_segmented = img * mask2[:, :, np.newaxis]

    # Clothing detection (using a pre-trained YOLOv5 model)
    # model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    # results = model(img_segmented)

    cv2.imshow("Segmented Image", img_segmented)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return img_segmented

def try_on_clothing(user_image, clothing_image, keypoints):
    # Read and pre-process user and clothing images
    user_image = user_image
    clothing_image = clothing_image[:,:,::-1]  # Convert BGR to RGB for OpenCV functions
    mask = cv2.cvtColor(clothing_image, cv2.COLOR_BGR2GRAY)  # Extract clothing mask

    # Applying scaling on clothing:
    scale_factor = 1  
    clothing_image = cv2.resize(clothing_image, None, fx=scale_factor, fy=scale_factor)

    # Align clothing based on keypoints
    user_points = np.array(keypoints, dtype="float32")
    clothing_points = np.array([(0, 0), (clothing_image.shape[1], 0), 
                                (clothing_image.shape[1], clothing_image.shape[0]), 
                                (0, clothing_image.shape[0])], dtype="float32")
    transformation_matrix = cv2.getPerspectiveTransform(user_points, clothing_points)

    user_image = user_image.astype(np.uint8) 
    warped_clothing = cv2.warpPerspective(clothing_image, transformation_matrix*0.8, 
                                          (user_image.shape[1], user_image.shape[0]))

    # mask resize to match user image size
    mask = cv2.resize(mask, (user_image.shape[1], user_image.shape[0]))

    # Perform bitwise operations
    user_image = cv2.bitwise_and(user_image, user_image, mask=cv2.bitwise_not(mask))
    composite_image = cv2.addWeighted(user_image,0.4, warped_clothing,0.6,0)

    # Convert to uint8
    composite_image = composite_image.astype(np.uint8)

    # Return the composite image
    return composite_image

def pose_estimator(user_image_path):
    # Initialize MediaPipe pose estimation
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8) 

    # Load the user image
    image = cv2.imread(user_image_path)

    # Convert image to RGB format (MediaPipe expects RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform pose estimation
    results = pose.process(image)

    # Extract keypoints
    if results.pose_landmarks:
        keypoints = results.pose_landmarks.landmark
        # Access keypoints for further processing (e.g., using coordinates for alignment)

    # Draw keypoints on the image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert back to BGR for OpenCV display
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow("Pose Estimation", image)
    cv2.waitKey(0)
    return keypoints, mp_pose

def align_and_convert_color(clothing_image_path, user_image):
    # Resize clothing image to be larger than user image
    clothing_image = cv2.imread(clothing_image_path)
    scale_factor = 1
    clothing_image = cv2.resize(clothing_image, None, fx=scale_factor, fy=scale_factor)
    # clothing_image = cv2.resize(clothing_image, None, fx=user_image.shape[0], fy=user_image.shape[1])

    # Convert clothing images to grayscale
    clothing_gray = cv2.cvtColor(clothing_image, cv2.COLOR_BGR2GRAY)

    # Find contours in clothing image
    contours, _ = cv2.findContours(clothing_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)

        # Get bounding box of largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop clothing image to bounding box
        clothing_image = clothing_image[y:y+h, x:x+w]

    # Convert clothing image to same color space as user image
    clothing_image = cv2.cvtColor(clothing_image, cv2.COLOR_BGR2RGB)

    return clothing_image

user_image_path = r"\lady.jpeg"
clothing_image_path = r"\clothing.jpg"

#Another set of input images
# user_image_path = r"\Men_User_Image.jpg" #replace with the relative path
# clothing_image_path = r"\Tshirt_Clothing.jpg"

# Getting segmented clothing and user images
user_image = segmentor(user_image_path, image_type='user_image')
clothing_image = align_and_convert_color(clothing_image_path, user_image)
clothing_image = segmentor(clothing_image, image_type='clothing_image')

# getting body keypoints
body_keypoints, mp_pose = pose_estimator(user_image_path)

# getting keypoints of specific body parts to align the clothing image to user image
left_shoulder = body_keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER]
right_shoulder = body_keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER]
left_foot = body_keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]
right_foot = body_keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]

# making the composite image
keypoints = np.array([(0-60, 0-70), (clothing_image.shape[1]-60, 0-70), 
                                (clothing_image.shape[1]-60, clothing_image.shape[0]-70), 
                                (0-60, clothing_image.shape[0]-70)], dtype="float32")
# keypoints = [(100-50, 150-50), (350-50, 150-50), (350-50, 400-50), (100-50, 400-50)]
# keypoints = [(right_shoulder.x*100, right_shoulder.y*100), (left_shoulder.x*100, left_shoulder.y*100),
#              (right_foot.x*100, right_foot.y*100), (left_foot.x*100, left_foot.y*100)]
composite_image = try_on_clothing(user_image, clothing_image, keypoints)

cv2.imshow("Virtual Try-On", composite_image[0])
cv2.waitKey(0)
cv2.destroyAllWindows()